In [ ]:
!pip install torch==1.4.0+cu100 torchvision==0.5.0+cu100 -f https://download.pytorch.org/whl/torch_stable.html
!pip install torch-geometric \
  torch-sparse==latest+cu100 \
  torch-scatter==latest+cu100 \
  torch-cluster==latest+cu100 \
  -f https://pytorch-geometric.com/whl/torch-1.4.0.html

In [ ]:

import numpy as np
import pandas as pd
import sys, os
from random import shuffle
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Sequential, Linear, ReLU
from torch_geometric.nn import GINConv, global_add_pool
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
from models.gat import GATNet
from models.gat_gcn import GAT_GCN
from models.gcn import GCNNet
from models.ginconv import GINConvNet
from utils import FocalLoss
from utils import *
import datetime
import argparse

store_path = "/content/drive/MyDrive/05.New_DRP3_ge_meth/Pretrain_DRP/DRP_RS_ge_meth/"
data_path = "/content/drive/MyDrive/05.New_DRP3_ge_meth/recall_moli/create_data/Splitting_data_new/ge_meth/" # data to process
data_processed_path = "/content/drive/MyDrive/05.New_DRP3_ge_meth/recall_moli/create_data/Splitting_data_new/ge_meth/processed/" # data processed
pretrain_model_path = "/content/drive/MyDrive/05.New_DRP3_ge_meth/Pretrain_DRP/DRP_RS_ge_meth/model_GINconvNet_GDSC_ge_meth_new.model" # weight of the model (GIN)
# GINConv model
class GINConvNet(torch.nn.Module):
    def __init__(self, n_output=1,num_features_xd=78, num_features_xt=25,
                 n_filters=32, embed_dim=128, output_dim=128, dropout=0.2):

        super(GINConvNet, self).__init__()

        dim = 32
        self.dropout = nn.Dropout(dropout)
        self.relu = nn.ReLU()
        self.n_output = n_output
        # convolution layers
        nn1 = Sequential(Linear(num_features_xd, dim), ReLU(), Linear(dim, dim))
        self.conv1 = GINConv(nn1)
        self.bn1 = torch.nn.BatchNorm1d(dim)

        nn2 = Sequential(Linear(dim, dim), ReLU(), Linear(dim, dim))
        self.conv2 = GINConv(nn2)
        self.bn2 = torch.nn.BatchNorm1d(dim)

        nn3 = Sequential(Linear(dim, dim), ReLU(), Linear(dim, dim))
        self.conv3 = GINConv(nn3)
        self.bn3 = torch.nn.BatchNorm1d(dim)

        nn4 = Sequential(Linear(dim, dim), ReLU(), Linear(dim, dim))
        self.conv4 = GINConv(nn4)
        self.bn4 = torch.nn.BatchNorm1d(dim)

        nn5 = Sequential(Linear(dim, dim), ReLU(), Linear(dim, dim))
        self.conv5 = GINConv(nn5)
        self.bn5 = torch.nn.BatchNorm1d(dim)

        self.fc1_xd = Linear(dim, output_dim)

        # 1D convolution on protein sequence  # mut features
        self.embedding_xt_mut = nn.Embedding(num_features_xt + 1, embed_dim)
        self.conv_xt_mut_1 = nn.Conv1d(in_channels=1000, out_channels=n_filters, kernel_size=8)

        # 1D convolution on protein sequence # mut features
        self.embedding_xt_meth = nn.Embedding(num_features_xt + 1, embed_dim)
        self.conv_xt_meth_1 = nn.Conv1d(in_channels=1000, out_channels=n_filters, kernel_size=8)


        # cell line mut feature
        self.conv_xt_mut_1 = nn.Conv1d(in_channels=1, out_channels=n_filters, kernel_size=8)
        self.pool_xt_mut_1 = nn.MaxPool1d(3)
        self.conv_xt_mut_2 = nn.Conv1d(in_channels=n_filters, out_channels=n_filters*2, kernel_size=8)
        self.pool_xt_mut_2 = nn.MaxPool1d(3)
        self.conv_xt_mut_3 = nn.Conv1d(in_channels=n_filters*2, out_channels=n_filters*4, kernel_size=8)
        self.pool_xt_mut_3 = nn.MaxPool1d(3)
        self.fc1_xt_mut = nn.Linear(1280, output_dim)

        # cell line meth feature
        self.conv_xt_meth_1 = nn.Conv1d(in_channels=1, out_channels=n_filters, kernel_size=8)
        self.pool_xt_meth_1 = nn.MaxPool1d(3)
        self.conv_xt_meth_2 = nn.Conv1d(in_channels=n_filters, out_channels=n_filters*2, kernel_size=8)
        self.pool_xt_meth_2 = nn.MaxPool1d(3)
        self.conv_xt_meth_3 = nn.Conv1d(in_channels=n_filters*2, out_channels=n_filters*4, kernel_size=8)
        self.pool_xt_meth_3 = nn.MaxPool1d(3)
        self.fc1_xt_meth = nn.Linear(83584, output_dim)

        # combined layers
        self.fc1 = nn.Linear(3*output_dim, 1024)
        self.fc2 = nn.Linear(1024, 128)
        self.out = nn.Linear(128, n_output)

        # activation and regularization
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        #print(x)
        #print(data.target)
        x = F.relu(self.conv1(x, edge_index))
        x = self.bn1(x)
        x = F.relu(self.conv2(x, edge_index))
        x = self.bn2(x)
        x = F.relu(self.conv3(x, edge_index))
        x = self.bn3(x)
        x = F.relu(self.conv4(x, edge_index))
        x = self.bn4(x)
        x = F.relu(self.conv5(x, edge_index))
        x = self.bn5(x)
        x = global_add_pool(x, batch)
        x = F.relu(self.fc1_xd(x))
        x = F.dropout(x, p=0.2, training=self.training)

        # protein input feed-forward:
        target_mut = data.target_mut
        #print(data.target_mut)
        #print(len(data.target_mut))
        #print(len(data.target_mut[1]))
        target_mut = target_mut[:,None,:]

        # protein input feed-forward:
        target_meth = data.target_meth
        #print(data.target_meth)
        #print(len(data.target_meth))
        #print(len(data.target_meth[1]))
        target_meth = target_meth[:,None,:]


       
        # 1d conv layers xt_mut
        conv_xt_mut = self.conv_xt_mut_1(target_mut)
        conv_xt_mut = F.relu(conv_xt_mut)
        conv_xt_mut = self.pool_xt_mut_1(conv_xt_mut)
        conv_xt_mut = self.conv_xt_mut_2(conv_xt_mut)
        conv_xt_mut = F.relu(conv_xt_mut)
        conv_xt_mut = self.pool_xt_mut_2(conv_xt_mut)
        conv_xt_mut = self.conv_xt_mut_3(conv_xt_mut)
        conv_xt_mut = F.relu(conv_xt_mut)
        conv_xt_mut = self.pool_xt_mut_3(conv_xt_mut)
        
        # 1d conv layers
        conv_xt_meth = self.conv_xt_meth_1(target_meth)
        conv_xt_meth = F.relu(conv_xt_meth)
        conv_xt_meth = self.pool_xt_meth_1(conv_xt_meth)
        conv_xt_meth = self.conv_xt_meth_2(conv_xt_meth)
        conv_xt_meth = F.relu(conv_xt_meth)
        conv_xt_meth = self.pool_xt_meth_2(conv_xt_meth)
        conv_xt_meth = self.conv_xt_meth_3(conv_xt_meth)
        conv_xt_meth = F.relu(conv_xt_meth)
        conv_xt_meth = self.pool_xt_meth_3(conv_xt_meth)


        # flatten mut
        xt_mut = conv_xt_mut.view(-1, conv_xt_mut.shape[1] * conv_xt_mut.shape[2])
        xt_mut = self.fc1_xt_mut(xt_mut)
        #print(xt_mut)
        
        # flatten meth
        xt_meth = conv_xt_meth.view(-1, conv_xt_meth.shape[1] * conv_xt_meth.shape[2])
        xt_meth = self.fc1_xt_meth(xt_meth)
        #print(xt_meth)
        

        # concat
        xc = torch.cat((x, xt_mut, xt_meth), 1)
        # add some dense layers
        xc = self.fc1(xc)
        xc = self.relu(xc)
        xc = self.dropout(xc)
        xc = self.fc2(xc)
        xc = self.relu(xc)
        xc = self.dropout(xc)
        out = self.out(xc)
        return out, x
from sklearn.metrics import roc_auc_score
def roc_auc_compute_fn(y_preds: torch.Tensor, y_targets: torch.Tensor) -> float:
    y_true = y_targets.numpy()
    y_pred = y_preds.numpy()
    return roc_auc_score(y_true, y_pred)

def set_parameter_requires_grad(model, feature_extracting=True):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

# training function at each epoch
def train(model, device, train_loader, optimizer, epoch, log_interval):
    print('Training on {} samples...'.format(len(train_loader.dataset)))
    model.train()
    loss_fn = nn.BCELoss()
    avg_loss = []
    for batch_idx, data in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        output, _ = model(data)
        loss = loss_fn(output, data.y.view(-1,2).float().to(device))
        loss.backward()
        optimizer.step()
        avg_loss.append(loss.item())
        if batch_idx % log_interval == 0:
            print('Train epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch,
                                                                           batch_idx * len(data.x),
                                                                           len(train_loader.dataset),
                                                                           100. * batch_idx / len(train_loader),
                                                                           loss.item()))
    return sum(avg_loss)/len(avg_loss)

def predicting(model, device, loader):
    model.eval()
    total_preds = torch.Tensor()
    total_labels = torch.Tensor()
    print('Make prediction for {} samples...'.format(len(loader.dataset)))
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            output, _ = model(data)
            total_preds = torch.cat((total_preds, output.cpu()), 0)
            total_labels = torch.cat((total_labels, data.y.view(-1, 2).cpu()), 0)
    return total_labels,total_preds




In [ ]:
lr = 0.001
num_epoch = 100
train_batch = 128
val_batch = 128
test_batch = 128 
log_interval = 20
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
#define model
n_class = 2
modeling = GINConvNet
model = modeling().to(device)

#load model 
model.load_state_dict(torch.load(pretrain_model_path))
# set_parameter_requires_grad(model)
# model.eval()
#add softmax
model.out = nn.Sequential(nn.Linear(128, 64), nn.ReLU(), nn.Linear(64, 32), nn.ReLU(), nn.Linear(32, n_class), nn.Softmax())
model = model.to(device)

cuda


In [ ]:
# test_data = TestbedDataset(root=data_path, dataset=dataset+'_test_mix')

#Train model

In [ ]:
model.eval()

GINConvNet(
  (dropout): Dropout(p=0.5, inplace=False)
  (relu): ReLU()
  (conv1): GINConv(nn=Sequential(
    (0): Linear(in_features=78, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
  ))
  (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): GINConv(nn=Sequential(
    (0): Linear(in_features=32, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
  ))
  (bn2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): GINConv(nn=Sequential(
    (0): Linear(in_features=32, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
  ))
  (bn3): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): GINConv(nn=Sequential(
    (0): Linear(in_features=32, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_feature

In [ ]:
print('Learning rate: ', lr)
print('Epochs: ', num_epoch)
feature_extract = True
model_st = "ge_meth_cls"
dataset = 'GDSC'
train_losses = []
val_losses = []
val_pearsons = []
print('\nrunning on ', model_st + '_' + dataset )
processed_data_file_train = data_processed_path + dataset + '_train_mix'+'.pt'
processed_data_file_val = data_processed_path + dataset + '_val_mix'+'.pt'
processed_data_file_test = data_processed_path + dataset + '_test_mix'+'.pt'
if ((not os.path.isfile(processed_data_file_train)) or (not os.path.isfile(processed_data_file_val)) or (not os.path.isfile(processed_data_file_test))):
    print('please run create_data.py to prepare data in pytorch format!')
else:
    train_data = TestbedDataset(root=data_path, dataset=dataset+'_train_mix')
    val_data = TestbedDataset(root=data_path, dataset=dataset+'_val_mix')
    test_data = TestbedDataset(root=data_path, dataset=dataset+'_test_mix')

    # make data PyTorch
    # mini-batch processing ready
    train_loader = DataLoader(train_data, batch_size=train_batch, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=val_batch, shuffle=False)
    test_loader = DataLoader(test_data, batch_size=test_batch, shuffle=False)
    print("CPU/GPU: ", torch.cuda.is_available())
            
    # training the model
    params_to_update = model.parameters()
    print("Params to learn:")
    if feature_extract:
        params_to_update = []
        for name,param in model.named_parameters():
            if param.requires_grad == True:
                params_to_update.append(param)
                print("\t",name)
    else:
        for name,param in model.named_parameters():
            if param.requires_grad == True:
                print("\t",name)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)
    bce = nn.BCELoss()
    optimizer = torch.optim.Adam(params_to_update, lr=lr)    
    # best_bce = 1000
    # best_pearson = 1
    # best_epoch = -1
    # model_file_name = store_path+'model_' + model_st + '_' + dataset +  '.model'
    # result_file_name = store_path+'result_' + model_st + '_' + dataset +  '.csv'
    # loss_fig_name = store_path+'model_' + model_st + '_' + dataset + '_loss'
    # pearson_fig_name = store_path+'model_' + model_st + '_' + dataset + '_pearson'
    

In [ ]:
best_bce = 1000
best_auc = 0
best_pearson = 1
best_epoch = -1
model_file_name = store_path+'model_' + model_st + '_' + dataset +  '.model'
result_file_name = store_path+'result_' + model_st + '_' + dataset +  '.csv'
loss_fig_name = store_path+'model_' + model_st + '_' + dataset + '_loss'
pearson_fig_name = store_path+'model_' + model_st + '_' + dataset + '_pearson'

In [ ]:
num_epoch = 200
best_ret_test = None
for epoch in range(num_epoch):
      train_loss = train(model, device, train_loader, optimizer, epoch+1, log_interval)
      G,P = predicting(model, device, val_loader)
      # G,P = predicting(model, device, test_loader)

      ret = [roc_auc_score(G,P),bce(G,P),pearson(G,P),spearman(G,P)]
                  
      G_test,P_test = predicting(model, device, test_loader)
      ret_test = [roc_auc_score(G_test,P_test),bce(G_test,P_test),pearson(G_test,P_test),spearman(G_test,P_test)]

      train_losses.append(train_loss)
      val_losses.append(ret[1])
      val_pearsons.append(ret[2])
      if ret[0]>best_auc:
          torch.save(model.state_dict(), model_file_name)
          with open(result_file_name,'w') as f:
              f.write(','.join(map(str,ret_test)))
          best_epoch = epoch+1
          best_bce = ret[1]
          best_pearson = ret[2]
          best_auc = ret[0]
          best_ret_test = ret_test
          print(' bce improved at epoch ', best_epoch, '; best_bce:', best_bce,model_st,dataset, "; best_auc:", best_auc)
      else:
          print(' no improvement since epoch ', best_epoch, '; best_bce, best pearson:', best_bce, best_pearson, model_st, dataset, "; best_auc:", best_auc)
draw_loss(train_losses, val_losses, loss_fig_name)
draw_pearson(val_pearsons, pearson_fig_name)
print(best_ret_test)

#Test on drugs

In [ ]:
pretrain_cls = "/content/drive/MyDrive/05.New_DRP3_ge_meth/Pretrain_DRP/DRP_RS_ge_meth/model_ge_meth_cls_GDSC.model"

In [ ]:
model.load_state_dict(torch.load(pretrain_cls))

<All keys matched successfully>

In [ ]:
data_path = "/content/drive/MyDrive/05.New_DRP3_ge_meth/MOLI/preprocess/processed_ge_meth/" # data to process

In [ ]:
GDSC_Docetaxel_test_mix = TestbedDataset(root=data_path, dataset="GDSC_Docetaxel_test_mix")
GDSC_Erlotinib_test_mix = TestbedDataset(root=data_path, dataset="GDSC_Erlotinib_test_mix")
GDSC_Gemcitabine_test_mix = TestbedDataset(root=data_path, dataset="GDSC_Gemcitabine_test_mix")
GDSC_Paclitaxel_test_mix = TestbedDataset(root=data_path, dataset="GDSC_Paclitaxel_test_mix")

GDSC_Docetaxel_test_mix = DataLoader(GDSC_Docetaxel_test_mix, batch_size=test_batch, shuffle=False)
GDSC_Erlotinib_test_mix = DataLoader(GDSC_Erlotinib_test_mix, batch_size=test_batch, shuffle=False)
GDSC_Gemcitabine_test_mix = DataLoader(GDSC_Gemcitabine_test_mix, batch_size=test_batch, shuffle=False)
GDSC_Paclitaxel_test_mix = DataLoader(GDSC_Paclitaxel_test_mix, batch_size=test_batch, shuffle=False)

test_loaders = [GDSC_Docetaxel_test_mix, GDSC_Erlotinib_test_mix, GDSC_Gemcitabine_test_mix, GDSC_Paclitaxel_test_mix]

Pre-processed data found: /content/drive/MyDrive/05.New_DRP3_ge_meth/MOLI/preprocess/processed_ge_meth/processed/GDSC_Docetaxel_test_mix.pt, loading ...
Pre-processed data found: /content/drive/MyDrive/05.New_DRP3_ge_meth/MOLI/preprocess/processed_ge_meth/processed/GDSC_Erlotinib_test_mix.pt, loading ...
Pre-processed data found: /content/drive/MyDrive/05.New_DRP3_ge_meth/MOLI/preprocess/processed_ge_meth/processed/GDSC_Gemcitabine_test_mix.pt, loading ...
Pre-processed data found: /content/drive/MyDrive/05.New_DRP3_ge_meth/MOLI/preprocess/processed_ge_meth/processed/GDSC_Paclitaxel_test_mix.pt, loading ...


In [ ]:
bce = nn.BCELoss()

In [ ]:
for test_loader in test_loaders:
  G_test,P_test = predicting(model, device, test_loader)
  ret_test = [roc_auc_score(G_test,P_test),bce(G_test,P_test),pearson(G_test,P_test),spearman(G_test,P_test)]
  print(ret_test)